In [ ]:
from pathlib import Path
import numpy as np
import cv2
import matplotlib.pyplot as plt
from src.paths import DataPaths

def plot_background_subtraction(image_name: str, root: str = 'data', stretch_percentile: float = 99.0):
    '''
    Show the processed grayscale image beside blurred background and normalized difference.
    '''
    paths = DataPaths.from_root(root)
    proc_path = paths.processed / image_name
    bg_path = paths.root / 'background' / image_name
    if not proc_path.exists():
        raise FileNotFoundError(f'Processed image not found: {proc_path}')
    if not bg_path.exists():
        raise FileNotFoundError(f'Background file not found: {bg_path}')

    processed = cv2.imread(str(proc_path), cv2.IMREAD_GRAYSCALE)
    background = cv2.imread(str(bg_path), cv2.IMREAD_GRAYSCALE)
    if processed is None or background is None:
        raise IOError('Failed to load one of the images.')
    if processed.shape != background.shape:
        background = cv2.resize(background, (processed.shape[1], processed.shape[0]))

    diff = processed.astype(np.float32) - background.astype(np.float32)
    limit = np.percentile(np.abs(diff), stretch_percentile)
    limit = max(limit, 1.0)
    diff = np.clip(diff / limit, -1.0, 1.0)

    fig, axes = plt.subplots(1, 3, figsize=(14, 4))
    axes[0].imshow(processed, cmap='gray')
    axes[0].set_title('Processed (grayscale)')
    axes[0].axis('off')
    axes[1].imshow(background, cmap='gray')
    axes[1].set_title('Blurred background')
    axes[1].axis('off')
    im = axes[2].imshow(diff, cmap='seismic', vmin=-1, vmax=1)
    axes[2].set_title('Difference (normalized)')
    axes[2].axis('off')
    fig.colorbar(im, ax=axes[2], orientation='vertical', shrink=0.9)
    plt.tight_layout()
    return fig, axes


In [ ]:
# Example usage
plot_background_subtraction('0_10.jpg')
